# **Instructions**:

### 1 - Open your game that you want to perform detections
### 2 - In the game window, get the name of it's title bar 
### 3 - Update the variable "window_name" with the game title bar name
### 4 - Run all cells to start detecting objects using your trained model

In [1]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os
import random

In [2]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)
    
    def get_window_size(self):
        return (self.w, self.h)

In [3]:
class ImageProcessor:
    W = 0
    H = 0
    net = None
    ln = None
    classes = {}
    colors = []
    image = None

    def __init__(self, img_size, cfg_file, weights_file):
        np.random.seed(42)
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i-1] for i in self.net.getUnconnectedOutLayers()]
        self.W = img_size[0]
        self.H = img_size[1]
        self.image = None
        
        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()
        
        # If you plan to utilize more than six classes, please include additional colors in this list.
        self.colors = [
            (0, 0, 255), 
            (0, 255, 0), 
            (255, 0, 0), 
            (255, 255, 0), 
            (255, 0, 255), 
            (0, 255, 255)
        ]
        

    def proccess_image(self, img):

        blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)
        
        coordinates = self.get_coordinates(outputs, 0.5)

        self.image = self.draw_identified_objects(img, coordinates)
        
        return coordinates

    def get_coordinates(self, outputs, conf):

        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w//2), int(y - h//2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            #cut player box from coordinates
            if self.classes[classIDs[i]] != 'player':
            #cut player box from coordinates
                coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})
        return coordinates

    def draw_identified_objects(self, img, coordinates):
        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']
            color = self.colors[classID]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv.imshow('window',  img)
        return img
    def get_processed_image(self):
        return self.image


In [4]:
import keyboard
def run_from_pigs(coordinates, center_x, center_y, img):
    
    
    final_x = 0
    final_y = 0
    for c in coordinates:
        if c["class"] == 0:
            final_x += (center_x - c['x'])
            final_y += (center_y - c['y'])
    
    if final_x != 0 and final_y != 0:
        final_x += center_x
        final_y += center_y
    print(f"final_x:{final_x}final_y:{final_y}")
    if final_x == 0 and final_y == 0:
        return 's'
    #line
    cv.line(img, (final_x, final_y),(center_x, center_y), (0, 0, 255), 3)
    #line
    cv.imshow('window',  img)
    
    if final_x > center_x and final_y > center_y:
        if final_x < center_x * 1.5 and final_y > center_y * 1.5:
            return 'x'
        elif final_x > center_x * 1.5 and final_y < center_y * 1.5:
            return 'd'
        else:
            return 'c'
    elif final_x < center_x and final_y < center_y:
        if final_x < center_x * 0.5 and final_y > center_y * 0.5:
            return 'a'
        elif final_x > center_x * 0.5 and final_y < center_y * 0.5:
            return 'w'
        else:
            return 'q'
    elif final_x > center_x and final_y < center_y:
        if final_x < center_x * 1.5 and final_y < center_y * 0.5:
            return 'w'
        elif final_x > center_x * 1.5 and final_y > center_y * 0.5:
            return 'd'
        else:
            return 'e'
    elif final_x < center_x and final_y > center_y:
        if final_x < center_x * 0.5 and final_y < center_y * 1.5:
            return 'a'
        elif final_x > center_x * 0.5 and final_y > center_y * 1.5:
            return 'x'
        else:
            return 'z'
    else:
        return 'error'
     

def release_except(keys_to_not_release):
    keys_to_release = ['w','a','s','d']
    for k in keys_to_release:
        if k not in keys_to_not_release:
            keyboard.release(k)
    

In [5]:
# Run this cell to initiate detections using the trained model.

window_name = "Don't Starve Together"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)
while(True):
    
    ss = wincap.get_screenshot()
    
    if cv.waitKey(1) == ord('y'):
        cv.destroyAllWindows()
        break

    coordinates = improc.proccess_image(ss)
    
    for coordinate in coordinates:
        print(coordinate)
    if keyboard.is_pressed('u'):
        match run_from_pigs(coordinates, 500, 250, improc.get_processed_image()):
            case 'q':
                release_except(['w','a'])
                keyboard.press('w')
                keyboard.press('a')
            case 'w':
                release_except(['w'])
                keyboard.press('w')
            case 'e':
                release_except(['w','d'])
                keyboard.press('w')
                keyboard.press('d')
            case 'a':
                release_except(['a'])
                keyboard.press('a')
            case 'd':
                release_except(['d'])
                keyboard.press('d')
            case 'z':
                release_except(['a','s'])
                keyboard.press('a')
                keyboard.press('s')
            case 'x':
                release_except(['s'])
                keyboard.press('s')
            case 'c':
                release_except(['d','s'])
                keyboard.press('d')
                keyboard.press('s')
            case 's':
                release_except([])
            case _:
                release_except([])
                print("failure in function run_from_pigs, key press not detected")
    else:
        release_except([])
            
    
    
    # If you have limited computer resources, consider adding a sleep delay between detections.
    #sleep(0.2)

print('Finished.')

C:\Users\lukas\AppData\Local\Temp\ipykernel_14800\2781964491.py:32: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


{'x': 476, 'y': 243, 'w': 42, 'h': 71, 'class': np.int64(0), 'class_name': 'pig'}
{'x': 466, 'y': 223, 'w': 44, 'h': 68, 'class': np.int64(0), 'class_name': 'pig'}
{'x': 770, 'y': 14, 'w': 31, 'h': 23, 'class': np.int64(2), 'class_name': 'tentacle'}
{'x': 773, 'y': 14, 'w': 31, 'h': 26, 'class': np.int64(2), 'class_name': 'tentacle'}
{'x': 805, 'y': 38, 'w': 31, 'h': 22, 'class': np.int64(2), 'class_name': 'tentacle'}
{'x': 809, 'y': 38, 'w': 32, 'h': 25, 'class': np.int64(2), 'class_name': 'tentacle'}
{'x': 814, 'y': 43, 'w': 32, 'h': 26, 'class': np.int64(2), 'class_name': 'tentacle'}
{'x': 485, 'y': 0, 'w': 37, 'h': 12, 'class': np.int64(1), 'class_name': 'pig_house'}
{'x': 484, 'y': 1, 'w': 38, 'h': 17, 'class': np.int64(1), 'class_name': 'pig_house'}
{'x': 482, 'y': 1, 'w': 42, 'h': 25, 'class': np.int64(1), 'class_name': 'pig_house'}
{'x': 484, 'y': 2, 'w': 41, 'h': 28, 'class': np.int64(1), 'class_name': 'pig_house'}
{'x': 486, 'y': 2, 'w': 40, 'h': 40, 'class': np.int64(0), 'cl